## Обучение модели классификации для цветков Iris

In [1]:
!pip install pyspark

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StringType, IntegerType, ArrayType, DoubleType, BooleanType

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import warnings
warnings.filterwarnings("ignore")

sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
# 1-2. Загружаем данные из файла iris.csv

df = spark.read.options(sep=",", quote='', header=True).csv('iris.csv')

In [5]:
df.show(5)

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
+-------------+---------------+----------------+---------------+------------+
only showing top 5 rows



In [6]:
# Создаем новый датафрейм из предыдущего, в котором будем использовать новые имена для колонок

newColumns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']
df_new = df.toDF(*newColumns)

In [7]:
# Удаляем кавычки из значений в первой и последней колонках 

df_new = df_new.withColumn('sepal_length', regexp_replace(col('sepal_length'), '"', ''))\
               .withColumn('variety', regexp_replace(col('variety'), '"', ''))

In [8]:
# Проверяем схему нашего нового датафрейма.

df_new.printSchema()

root
 |-- sepal_length: string (nullable = true)
 |-- sepal_width: string (nullable = true)
 |-- petal_length: string (nullable = true)
 |-- petal_width: string (nullable = true)
 |-- variety: string (nullable = true)



In [9]:
# Задаем корректные типы данных для числовых значений

df_new = df_new\
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType()))\
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType()))\
    .withColumn('petal_length', col('petal_length').cast(DoubleType()))\
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [10]:
# Выводим 5 первых строк данных

df_new.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [13]:
# 3. При помощи VectorAssembler преобразовываем все колонки с признаками в одну

pipeline = Pipeline(stages = [StringIndexer(inputCol = 'variety', outputCol = 'varietyInd'),
                   VectorAssembler(inputCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol = 'Features')])

In [14]:
# Подгоняем через заданный pipeline наш датасет

pipelineTrained = pipeline.fit(df_new)

In [15]:
# Применяем метод transform для объекта VectorAssembler, который преобразует наш датафрейм

pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [16]:
# С новым признаком Features создаем новый датафрейм

df_features = pipelineTrained.transform(df_new)

In [23]:
# 4. Разбиваем данные на train и test.
# При этом назначаем соответствующие веса и указываем seed для сохранения рандомного разделения выборки

train, test = df_features.randomSplit([0.8, 0.2], seed = 35)

In [28]:
train.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.

In [29]:
# 5. Создаем модель логистической регрессии и обучаем ее

lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

In [31]:
# Получаем предсказания с помощью метода transform

train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [32]:
# Предсказания на обучающей выборке

train_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[55.1497011816620...|[1.0,4.7302496569...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[46.5966098393767...|[1.0,1.5895428182...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[50.2591019033602...|[1.0,1.3995721418...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[55.7595478673163...|[1.0,6.9527164006.

In [33]:
# Предсказания на тестовой выборке

test_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.6,1.0,0.2]|[67.6705948996092...|[1.0,1.9335472491...|       0.0|
|         4.7|        3.2|         1.6|        0.2|    Setosa|       0.0|[4.7,3.2,1.6,0.2]|[50.2827624088470...|[1.0,8.8135608877...|       0.0|
|         4.9|        2.4|         3.3|        1.0|Versicolor|       1.0|[4.9,2.4,3.3,1.0]|[-6.2761283304909...|[2.83853063388669...|       1.0|
|         4.9|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.9,3.1,1.5,0.2]|[46.6181563672388...|[1.0,2.8553985098.

In [36]:
# 6. Воспользуемся MulticlassClassificationEvaluator для оценки качества на train и test множестве

ev = MulticlassClassificationEvaluator(labelCol = 'varietyInd')

In [39]:
print('Точность предсказания на тестовой выборке:', ev.evaluate(test_res)*100, '%')

Точность предсказания на тестовой выборке: 100.0 %


In [38]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на обучающей выборке: 98.23008849557523 %


### Выводы: по результату проведенной работы можно сделать вывод, что модель логистической регрессии для наших данных довольно хорошо подошла для классификации, точность предсказания на тестовой выборке составила 100%, на обучающей - 98 %.